<a href="https://colab.research.google.com/github/krmonline/AFSCTraining/blob/master/Web_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pythainlp > /dev/null
!pip install deepcut > /dev/null

In [10]:
import pandas as pd
import re
import collections
import numpy as np
from requests import get
from bs4 import BeautifulSoup
from urllib.parse import quote, unquote
import datetime as dt
import deepcut
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag 
from pythainlp.tag.named_entity import ThaiNameTagger

In [21]:
def def_province(x):
    province  = ['เชียงราย', 'เชียงใหม่', 'น่าน', 'พะเยา', 'แพร่', 'แม่ฮ่องสอน', 'ลำปาง', 'ลำพูน', 'อุตรดิตถ์', 'กาฬสินธุ์', 'ขอนแก่น', 'ชัยภูมิ', 'นครพนม', 'นครราชสีมา', 'บึงกาฬ', 'บุรีรัมย์', 'มหาสารคาม', 'มุกดาหาร', 'ยโสธร', 'ร้อยเอ็ด', 'เลย', 'สกลนคร', 'สุรินทร์', 'ศรีสะเกษ', 'หนองคาย', 'หนองบัวลำภู', 'อุดรธานี', 'อุบลราชธานี', 'อำนาจเจริญ', 'กำแพงเพชร', 'ชัยนาท', 'นครนายก', 'นครปฐม', 'นครสวรรค์', 'นนทบุรี', 'ปทุมธานี', 'พระนครศรีอยุธยา', 'พิจิตร', 'พิษณุโลก', 'เพชรบูรณ์', 'ลพบุรี', 'สมุทรปราการ', 'สมุทรสงคราม', 'สมุทรสาคร', 'สิงห์บุรี', 'สุโขทัย', 'สุพรรณบุรี', 'สระบุรี', 'อ่างทอง', 'อุทัยธานี', 'จันทบุรี', 'ฉะเชิงเทรา', 'ชลบุรี', 'ตราด', 'ปราจีนบุรี', 'ระยอง', 'สระแก้ว', 'กาญจนบุรี', 'ตาก', 'ประจวบคีรีขันธ์', 'เพชรบุรี', 'ราชบุรี', 'กระบี่', 'ชุมพร', 'ตรัง', 'นครศรีธรรมราช', 'นราธิวาส', 'ปัตตานี', 'พังงา', 'พัทลุง', 'ภูเก็ต', 'ระนอง', 'สตูล', 'สงขลา', 'สุราษฎร์ธานี', 'ยะลา', 'กรุงเทพมหานคร']
    row_location =  [i.strip() for i in x.split(',')]
    p_found = set([])
    for each_location in row_location:
        for  p in province:
            if re.search(p,each_location):
                p_found.add(p)
    return ",".join(list(p_found)).strip()

def def_country(x):
    country = ['รัสเซีย', 'แคนาดา', 'สหรัฐอเมริกา', 'จีน', 'บราซิล', 'ออสเตรเลีย', 'อินเดีย', 'อาร์เจนตินา', 'คาซัคสถาน', 'แอลจีเรีย', 'สาธารณรัฐประชาธิปไตยคองโก', 'กรีนแลนด์', 'ซาอุดีอาระเบีย', 'เม็กซิโก', 'อินโดนีเซีย', 'ซูดาน', 'ลิเบีย', 'อิหร่าน', 'มองโกเลีย', 'เปรู', 'ชาด', 'ไนเจอร์', 'แองโกลา', 'มาลี', 'แอฟริกาใต้', 'โคลอมเบีย', 'เอธิโอเปีย', 'โบลิเวีย', 'มอริเตเนีย', 'อียิปต์', 'แทนซาเนีย', 'ไนจีเรีย', 'เวเนซุเอลา', 'นามิเบีย', 'โมซัมบิก', 'ปากีสถาน', 'ตุรกี', 'ชิลี', 'แซมเบีย', 'พม่า', 'อัฟกานิสถาน', 'เซาท์ซูดาน', 'ฝรั่งเศส', 'โซมาเลีย', 'สาธารณรัฐแอฟริกากลาง', 'ยูเครน', 'มาดากัสการ์', 'บอตสวานา', 'เคนยา', 'เยเมน', 'ไทย', 'สเปน', 'เติร์กเมนิสถาน', 'แคเมอรูน', 'ปาปัวนิวกินี', 'สวีเดน', 'อุซเบกิสถาน', 'โมร็อกโก', 'อิรัก', 'ปารากวัย', 'ซิมบับเว', 'ญี่ปุ่น', 'เยอรมนี', 'สาธารณรัฐคองโก', 'ฟินแลนด์', 'เวียดนาม', 'มาเลเซีย', 'นอร์เวย์', 'โกตดิวัวร์', 'โปแลนด์', 'โอมาน', 'อิตาลี', 'ฟิลิปปินส์', 'บูร์กินาฟาโซ', 'นิวซีแลนด์', 'กาบอง', 'เวสเทิร์นสะฮารา', 'เอกวาดอร์', 'กินี', 'สหราชอาณาจักร', 'ยูกันดา', 'กานา', 'โรมาเนีย', 'ลาว', 'กายอานา', 'เบลารุส', 'คีร์กีซสถาน', 'เซเนกัล', 'ซีเรีย', 'กัมพูชา', 'อุรุกวัย', 'ตูนิเซีย', 'ซูรินาเม', 'เนปาล', 'บังกลาเทศ', 'ทาจิกิสถาน', 'กรีซ', 'นิการากัว', 'ประเทศเกาหลีเหนือ', 'มาลาวี', 'เอริเทรีย', 'เบนิน', 'ฮอนดูรัส', 'ไลบีเรีย', 'บัลแกเรีย', 'คิวบา', 'กัวเตมาลา', 'ไอซ์แลนด์', 'เกาหลีใต้', 'ฮังการี', 'โปรตุเกส', 'จอร์แดน', 'เฟรนช์เกียนา', 'เซอร์เบีย', 'อาเซอร์ไบจาน', 'ออสเตรีย', 'สหรัฐอาหรับเอมิเรตส์', 'สาธารณรัฐเช็ก', 'ปานามา', 'เซียร์ราลีโอน', 'ไอร์แลนด์', 'จอร์เจีย', 'ศรีลังกา', 'ลิทัวเนีย', 'ลัตเวีย', 'สฟาลบาร์', 'โตโก', 'โครเอเชีย', 'บอสเนียและเฮอร์เซโกวีนา', 'คอสตาริกา', 'สโลวาเกีย', 'สาธารณรัฐโดมินิกัน', 'ภูฏาน', 'เอสโตเนีย', 'เดนมาร์ก', 'เนเธอร์แลนด์', 'สวิตเซอร์แลนด์', 'กินี-บิสเซา', 'ไต้หวัน', 'มอลโดวา', 'เบลเยียม', 'เลโซโท', 'ประเทศอาร์เมเนีย', 'แอลเบเนีย', 'หมู่เกาะโซโลมอน', 'อิเควทอเรียลกินี', 'บุรุนดี', 'เฮติ', 'รวันดา', 'มาซิโดเนีย', 'จิบูตี', 'เบลีซ', 'เอลซัลวาดอร์', 'อิสราเอล', 'สโลวีเนีย', 'นิวแคลิโดเนีย', 'ฟิจิ', 'คูเวต', 'สวาซิแลนด์', 'ติมอร์-เลสเต', 'บาฮามาส', 'ประเทศมอนเตเนโกร', 'วานูอาตู', 'หมู่เกาะฟอล์กแลนด์', 'กาตาร์', 'แกมเบีย', 'จาเมกา', 'เลบานอน', 'ไซปรัส', 'เปอร์โตริโก', 'เฟรนช์เซาเทิร์นและแอนตาร์กติกเทร์ทอรีส์', 'เวสต์แบงก์', 'บรูไน', 'ตรินิแดดและโตเบโก', 'เฟรนช์โปลินีเซีย (ฝรั่งเศส)', 'เคปเวิร์ด', 'เกาะเซาท์จอร์เจียและหมู่เกาะเซาท์แซนด์วิช', 'ซามัว', 'ลักเซมเบิร์ก', 'เรอูว์นียง', 'คอโมโรส', 'มอริเชียส', 'กัวเดอลุป', 'หมู่เกาะแฟโร', 'มาร์ตีนิก', 'ฮ่องกง', 'ประเทศเซาตูเมและปรินซิปี', 'คิริบาส', 'โดมินิกา', 'ตองกา', 'ไมโครนีเซีย', 'สิงคโปร์', 'บาห์เรน', 'เซนต์ลูเซีย', 'เกาะแมน', 'กวม', 'หมู่เกาะนอร์เทิร์นมาเรียนา', 'อันดอร์รา', 'ปาเลา', 'เซเชลส์', 'คูราเซา', 'แอนติกาและบาร์บูดา', 'บาร์เบโดส', 'หมู่เกาะเติกส์และหมู่เกาะเคคอส', 'เกาะเฮิร์ดและหมู่เกาะแมกดอนัลด์', 'เซนต์เฮเลนา', 'เซนต์วินเซนต์และเกรนาดีนส์', 'มายอต', 'ยานไมเอน', 'ฉนวนกาซา', 'หมู่เกาะเวอร์จินของสหรัฐอเมริกา', 'เกรเนดา', 'มอลตา', 'มัลดีฟส์', 'หมู่เกาะวาลลิสและหมู่เกาะฟุตูนา', 'หมู่เกาะเคย์แมน', 'เซนต์คิตส์และเนวิส', 'นีอูเอ', 'แซงปีแยร์และมีเกอลง', 'หมู่เกาะคุก', 'อเมริกันซามัว', 'อารูบา', 'หมู่เกาะมาร์แชลล์', 'ลิกเตนสไตน์', 'หมู่เกาะบริติชเวอร์จิน', 'เกาะคริสต์มาส', 'เดเกเลีย', 'อาโกรตีรี', 'เจอร์ซีย์', 'แองกวิลลา', 'มอนต์เซอร์รัต', 'เกิร์นซีย์', 'ซานมารีโน', 'บริติชอินเดียนโอเชียนเทร์ริทอรี', 'เกาะบูเวต', 'เบอร์มิวดา', 'หมู่เกาะพิตแคร์น', 'เกาะนอร์ฟอล์ก', 'เกาะยูโรปา (ฝรั่งเศส)', 'ตูวาลู', 'มาเก๊า', 'นาอูรู', 'หมู่เกาะโคโคส (หมู่เกาะคีลิง)', 'แพลไมราอะทอลล์', 'โตเกเลา', 'เกาะเวก', 'หมู่เกาะมิดเวย์', 'เกาะคลิปเพอร์ตัน', 'เกาะนาวาสซา', 'เกาะแอชมอร์และเกาะคาร์เทียร์', 'หมู่เกาะโกลริโอโซ', 'หมู่เกาะสแปรตลี', 'เกาะจาร์วิส', 'เกาะฮวนเดโนวา', 'หมู่เกาะคอรัลซี', 'จอห์นสตันอะทอลล์', 'โมนาโก', 'เกาะฮาวแลนด์', 'เกาะเบเกอร์', 'คิงแมนรีฟ', 'เกาะตรอมแลง', 'นครรัฐวาติกัน', 'บัสซาสดาอินเดีย', 'หมู่เกาะพาราเซล']
    row_location =  [i.strip() for i in x.split(',')]
    c_found = set([])
    for each_location in row_location:
        for  c in country:
            if re.search(c,each_location):
                c_found.add(c)
    return ",".join(list(c_found)).strip()



def createJoin(context):
  for file in os.listdir(rootDir):
      filepath  = rootDir + file
      #print(filepath)
      dftmp = pd.read_csv(filepath,sep='|')
      dftmp['province'] = dftmp.location.fillna("").apply(def_province)
      dftmp['country'] = dftmp.location.fillna("").apply(def_country)
      dftmp['person'] = dftmp['person'].fillna("")
      dftmp['things']  = dftmp.content.apply(def_things)
      for i in dftmp.iterrows():
          k = i[0]
          v = i[1]
          nid = v.id
          for j in ['province','country','person','things']:
              dftmp2  = pd.DataFrame(columns=["nid","name"])
              ptmp = v[j]
              if ptmp != "":
                  #print(nid,province)
                  #print(ptmp)
                  for p in ptmp.split(","):
                      #print(nid,p)
                      dftmp2 = dftmp2.append({"nid":nid,"name":p.strip()},ignore_index=True)
              if(len(dftmp2) > 0):
                  #display(dftmp2)
                  fname_tmp = j +"_"+ file
                  filepath_fname_tmp = targetDir + fname_tmp
                  #display(filepath_fname_tmp)
                  #display(dftmp2)
                  dftmp2.to_csv(filepath_fname_tmp,sep="|",index=False)


In [11]:
past_day = 7
keyward = "สหรัฐ"

In [12]:
now = dt.datetime.now()
past_ts = now.timestamp() - 86400*past_day
past = dt.datetime.fromtimestamp(past_ts)
past_iso = past.isoformat()

In [37]:
header = []
array_news = []
months = {'ม.ค.':'01','ก.พ.':'02','มี.ค.':'03','เม.ย':'04','พ.ค.':'05','มิ.ย.':'06','ก.ค.':'07','ส.ค.':'08','ก.ย.':'09','ต.ค.':'10','พ.ย.':'11','ธ.ค.':'12'}
topics = {'สังคม':'social','การเมือง':'politic','อื่นๆ':'others'}
for j in range(0,20,10):
    #print(j)
    url = "https://search.posttoday.com/search/result?start="+str(j)+"&q="+keyward+"&category=news&sort=relevance&rows=10&xDateSearchRadio=&section=&xChannel=&xNewsSection=&xColumn=&columnistName=&topic=&people=&location=&searchedField=all&publishedDate=" + past_iso
    #print(url)
    r = get(url)
    content = BeautifulSoup(r.text).find_all('div', class_="detail")
    if len(content) < 1:
      break
    #print(content)
    for i in content:
        array_new = {}
        tmp = i.find_all('a')
        text = tmp[1].text
        if tmp[0].text == 'สังคม':
          topic = 'social'
        elif tmp[0].text == 'การเมือง':
            topic = 'politic'
        elif tmp[0].text == 'บันเทิง':
          topic = 'entertainment'
        else:
            topic = "others"
        link = tmp[1].get('href')
        mid = link.split('news')[-1]
        array_new["id"] = mid
        array_new["title"] = text
        if(text == "Online"): #Fix Bug
          continue
        print(text,link)
        #header.append()
        r2 = get(link)
        content2 = BeautifulSoup(r2.text).find('div', class_="article-content")
        #content2 = content2.replace("\n"," ")
        dt_thai = BeautifulSoup(r2.text).find('div', class_="date_time")
        arr_dt = dt_thai.text.split(" ")
        arr_new_dt = [str(int(arr_dt[3])-543),months[arr_dt[2]],arr_dt[1]]
        new_dt = " ".join(["-".join(arr_new_dt),arr_dt[5]+":00"])
        array_new['time'] = new_dt
        array_new['content'] = content2.text.replace("\n"," ").strip()
        array_new['tag'] = []
        array_new["link"] = link
        array_new["source"] = "posttoday"
        array_new['topic'] = topic
        #id|title|time|content|tag|link|source|topic|location|organization|person|phone|url|zip|things
        array_news.append(array_new)
df = pd.DataFrame(array_news)

บาทเปิด 29.90 บาทต่อดอลลาร์ ทรงตัว https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Ffinance-stock%2Fnews%2F645082&contentId=pt-news645082
ดาวโจนส์,น้ำมัน,ทองคำสหรัฐปิดบวก https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Ffinance-stock%2Fnews%2F645079&contentId=pt-news645079
11 บริษัทไทยติดอันดับ "Gold Class" ด้านความยั่งยืนสูงสุดในโลก https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Ffinance-stock%2Fstock%2F645058&contentId=pt-news645058
ผลทดสอบวัคซีนโควิดสัญชาติไทยในสัตว์กระตุ้นภูมิคุ้มกันได้ดี https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Fsocial%2Fgeneral%2F645054&contentId=pt-news645054
'บีโอไอ'ประคองลงทุนปี'64 ดันไม่ต่ำกว่า4-5 แสนล. https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Feconomy%2Fnews%2F644995&contentId=pt-news644995
IRPC ปัจจัยลบรุมปี'63 ขาดทุน 6,152 ลบ. ตั้งเป้า

In [38]:
df = pd.DataFrame(array_news)
df

,id,title,time,content,tag,link,source,topic
0,645082,บาทเปิด 29.90 บาทต่อดอลลาร์ ทรงตัว,2021-02-11 09:17:00,เงินบาทเปิดตลาด 29.90 บาทต่อดอลลาร์ ทรงตัว แนว...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
1,645079,"ดาวโจนส์,น้ำมัน,ทองคำสหรัฐปิดบวก",2021-02-11 06:55:00,หุ้นดาวโจนส์ปิดบวก 61.97 จุด หลังประธานเฟดส่งส...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
2,645058,"11 บริษัทไทยติดอันดับ ""Gold Class"" ด้านความยั่...",2021-02-10 18:54:00,"S&P Global เผยรายงาน ""The Sustainability Yearb...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
3,645054,ผลทดสอบวัคซีนโควิดสัญชาติไทยในสัตว์กระตุ้นภูมิ...,2021-02-10 18:39:00,องค์การเภสัชกรรม เผยผลวิจัยพัฒนาวัคซีนป้องกันโ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social
4,644995,'บีโอไอ'ประคองลงทุนปี'64 ดันไม่ต่ำกว่า4-5 แสนล.,2021-02-10 13:30:00,บีโอไอ ปักธงปี'64 หนุนลงทุนอุตฯการแพทย์ -ดิจิท...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
5,644984,"IRPC ปัจจัยลบรุมปี'63 ขาดทุน 6,152 ลบ. ตั้งเป้...",2021-02-10 11:54:00,IRPC ชี้ ไตรมาส 4 ปี'63 รายได้เริ่มขยับ ตามราค...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
6,644967,เด็กเพื่อไทยแนะบิ๊กตู่ นายกฯที่ดีต้องแก้ปากท้อ...,2021-02-10 10:04:00,เด็กเพื่อไทยแนะบิ๊กตู่การเป็นนายกฯที่ดีต้องแก้...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic
7,644965,บาทเปิด 29.91/95 บาทต่อดอลลาร์ แข็งค่าต่อเนื่อง,2021-02-10 09:32:00,เงินบาทเปิดตลาดวันนี้ 29.91/95 บาทต่อดอลลาร์ แ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
8,644950,"หุ้นสหรัฐฯ ปิดผสม ""ดาวโจนส์""ลด น้ำมัน,ทองคำบวก",2021-02-10 07:04:00,หุ้นดาวโจนส์ปิดลบ 9.93 จุด ด้านราคาน้ำมันเพิ่ม...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
9,644909,“อนุทิน” ฟุ้ง สถาบันโลวีจัดไทยเป็นดับ 4 คุมโคว...,2021-02-09 16:32:00,กระทรวงสาธารณสุข เผยสถาบันโลวี ออสเตรเลีย จัดใ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social


In [39]:
ner = ThaiNameTagger()

In [40]:
#id|title|time|content|tag|link|source|topic|location|organization|person|phone|url|zip|things
header2 = []
tags = [] 
arr_person = []
arr_date = []
arr_org = []
arr_location = []
arr_phone = []
arr_url = []
arr_zip = []
for i in df.content.values[:]:
    arr_text=word_tokenize(i, engine="deepcut")
    #p = pos_tag(arr_text,corpus='orchid_ud')
    #print(p)
    tmp = " ".join(arr_text)
    header2.append(tmp)
    tag = ner.get_ner(i,pos=False,tag=True)
    s = BeautifulSoup(tag)
    arr_person_tmp = []
    for i  in s.find_all("person"):
      person_tmp = i.contents[0].split('.')[-1]
      person_tmp = person_tmp.split('นาย')[-1]
      person_tmp = person_tmp.split('นาง')[-1]
      person_tmp = person_tmp.split('นางสาว')[-1]
      arr_person_tmp.append(person_tmp)
    person = ",".join(arr_person_tmp)
    arr_person.append(person)
    date = ",".join([i.contents[0] for i  in s.find_all("date")])
    arr_date.append(date)
    org = ",".join([i.contents[0] for i  in s.find_all("organization")])
    arr_org.append(org)
    location = ",".join([i.contents[0] for i  in s.find_all("location")])
    #print(location)
    arr_location.append(location)
    phone = ",".join([i.contents[0] for i  in s.find_all("phone")])
    arr_phone.append(phone)
    url = ",".join([i.contents[0] for i  in s.find_all("url")])
    arr_url.append(url)
    zip = ",".join([i.contents[0] for i  in s.find_all("zip")])
    arr_zip.append(zip)
#df['title_tk'] = header2
df['location'] = arr_location
df['organization'] = arr_org
df['person'] = arr_person
df['phone'] = arr_phone
df['url'] = arr_url
df['zip'] = arr_zip
#df

In [41]:
df['province'] = df.location.fillna("").apply(def_province)
df

,id,title,time,content,tag,link,source,topic,location,organization,person,phone,url,zip,province
0,645082,บาทเปิด 29.90 บาทต่อดอลลาร์ ทรงตัว,2021-02-11 09:17:00,เงินบาทเปิดตลาด 29.90 บาทต่อดอลลาร์ ทรงตัว แนว...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,สหรัฐ,ธนาคารกรุงศรีอยุธยา,,,,,
1,645079,"ดาวโจนส์,น้ำมัน,ทองคำสหรัฐปิดบวก",2021-02-11 06:55:00,หุ้นดาวโจนส์ปิดบวก 61.97 จุด หลังประธานเฟดส่งส...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,เมืองไทยประกันภัย,"หุ้นดาวโจนส์,ดาวโจนส์ตลาดหุ้นนิวยอร์ก,ธนาคารกล...","ดัชนีเอสแอนด์พี,เจอโรม พาวเวล,ไลต์สวีตครูด งวด...",,,,
2,645058,"11 บริษัทไทยติดอันดับ ""Gold Class"" ด้านความยั่...",2021-02-10 18:54:00,"S&P Global เผยรายงาน ""The Sustainability Yearb...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,"บ้านปู,ไทยออยล์,ไทยเบฟเวอเรจ,ประเทศสหรัฐอเมริก...","S&P,บริษัทไทย,ตลาดหลักทรัพย์แห่งประเทศไทย,บริษ...","ภากร ปีตธวัชชัย,ภากร",,,,
3,645054,ผลทดสอบวัคซีนโควิดสัญชาติไทยในสัตว์กระตุ้นภูมิ...,2021-02-10 18:39:00,องค์การเภสัชกรรม เผยผลวิจัยพัฒนาวัคซีนป้องกันโ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social,"ประเทศไทย,จ.สระบุรี,ประเทศไทย,ประเทศอินเดีย,ปร...","องค์การเภสัชกรรม,กระทรวงสาธารณสุข,(สธ.,องค์การ...","อนุทิน ชาญวีรกูล,พรรณี ปิติสุทธิธรรม หัวหน้าศู...",,,,สระบุรี
4,644995,'บีโอไอ'ประคองลงทุนปี'64 ดันไม่ต่ำกว่า4-5 แสนล.,2021-02-10 13:30:00,บีโอไอ ปักธงปี'64 หนุนลงทุนอุตฯการแพทย์ -ดิจิท...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,"ญี่ปุ่น,ไทย,ไทย,ประเทศโดยรวม,เกาหลีใต้,ประเทศญ...","บีโอไอ,คณะกรรมการส่งเสริมการลงทุน,ตลาดหลักทรัพ...","สาวดวงใจ อัศวจินตจิตร์,ประยุทธ์ จันทร์โอชา,ดวง...",,,,
5,644984,"IRPC ปัจจัยลบรุมปี'63 ขาดทุน 6,152 ลบ. ตั้งเป้...",2021-02-10 11:54:00,IRPC ชี้ ไตรมาส 4 ปี'63 รายได้เริ่มขยับ ตามราค...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,"จีน,ไทย,เอเชียตะวันออกเฉียงใต้","ปตท.ผลิตวัตถุดิบเพื่อผลิตหน้ากาก,บริษัท ไออาร์...","ชวลิต ทิพพาวนิช,นทราธิราช,ชวลิต",,,,
6,644967,เด็กเพื่อไทยแนะบิ๊กตู่ นายกฯที่ดีต้องแก้ปากท้อ...,2021-02-10 10:04:00,เด็กเพื่อไทยแนะบิ๊กตู่การเป็นนายกฯที่ดีต้องแก้...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic,"ไทย,ไทย,ไทย,ประเทศไทย,ไทย,สหรัฐ","เด็กเพื่อไทย,คณะอนุกรรมการด้านสิ่งแวดล้อมและทร...","วัฒนรักษ์ อำนรรฆสรเดช,ประยุทธ์ จันทร์โอชา,ประย...",,,,
7,644965,บาทเปิด 29.91/95 บาทต่อดอลลาร์ แข็งค่าต่อเนื่อง,2021-02-10 09:32:00,เงินบาทเปิดตลาดวันนี้ 29.91/95 บาทต่อดอลลาร์ แ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,"เขตเศรษฐกิจชายแดนใต้,สหรัฐ","ธนาคารกรุงเทพ,ธนาคารกลางสหรัฐ,สมาคมเศรษฐกิจแห่...",เงินบาทเช้านี้ เป็นไปตามทิศทางดอลลาร์ที่อ่อนค่...,,,,
8,644950,"หุ้นสหรัฐฯ ปิดผสม ""ดาวโจนส์""ลด น้ำมัน,ทองคำบวก",2021-02-10 07:04:00,หุ้นดาวโจนส์ปิดลบ 9.93 จุด ด้านราคาน้ำมันเพิ่ม...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,,"หุ้นดาวโจนส์,ดาวโจนส์ตลาดหุ้นนิวยอร์ก","สัญญาน้ำมันดิบเวสต์เทกซัส อินเตอร์มีเดียต,ไลต์...",,,,
9,644909,“อนุทิน” ฟุ้ง สถาบันโลวีจัดไทยเป็นดับ 4 คุมโคว...,2021-02-09 16:32:00,กระทรวงสาธารณสุข เผยสถาบันโลวี ออสเตรเลีย จัดใ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social,"ออสเตรเลีย,ประเทศไทย,ออสเตรเลีย,เวียดนาม,ไต้หว...","กระทรวงสาธารณสุข,สถาบันโลวี,กระทรวงสาธารณสุข,ส...","อนุทิน ชาญวีรกูล,อนุทิน",,,,


In [42]:
df.to_csv("news_admin.csv",sep='|',index=False)

In [ ]:
#id|title|time|content|tag|link|source|topic|location|organization|person|phone|url|zip|things
#df